0001196096 Antonelli Giacomo giacomo.antonelli4@studio.unibo.it

0001153900 Chevokin Nikita nikita.chevokin@studio.unibo.it

# Product Recognition of Books

## Image Processing and Computer Vision - Assignment Module \#1


Contacts:

- Prof. Giuseppe Lisanti -> giuseppe.lisanti@unibo.it
- Prof. Samuele Salti -> samuele.salti@unibo.it
- Alex Costanzino -> alex.costanzino@unibo.it
- Francesco Ballerini -> francesco.ballerini4@unibo.it

Computer vision-based object detection techniques can be applied in library or bookstore settings to build a system that identifies books on shelves.

Such a system could assist in:
* Helping visually impaired users locate books by title/author;
* Automating inventory management (e.g., detecting misplaced or out-of-stock books);
* Enabling faster book retrieval by recognizing spine text or cover designs.

## Task
Develop a computer vision system that, given a reference image for each book, is able to identify such book from one picture of a shelf.

<figure>
<a href="https://ibb.co/pvLVjbM5"><img src="https://i.ibb.co/svVx9bNz/example.png" alt="example" border="0"></a>
</figure>

For each type of product displayed on the shelf, the system should compute a bounding box aligned with the book spine or cover and report:
1. Number of instances;
1. Dimension of each instance (area in pixel of the bounding box that encloses each one of them);
1. Position in the image reference system of each instance (four corners of the bounding box that enclose them);
1. Overlay of the bounding boxes on the scene images.

<font color="red"><b>Each step of this assignment must be solved using traditional computer vision techniques.</b></font>

#### Example of expected output
```
Book 0 - 2 instance(s) found:
  Instance 1 {top_left: (100,200), top_right: (110, 220), bottom_left: (10, 202), bottom_right: (10, 208), area: 230px}
  Instance 2 {top_left: (90,310), top_right: (95, 340), bottom_left: (24, 205), bottom_right: (23, 234), area: 205px}
Book 1 – 1 instance(s) found:
.
.
.
```

## Data
Two folders of images are provided:
* Models: contains one reference image for each product that the system should be able to identify;
* Scenes: contains different shelve pictures to test the developed algorithm in different scenarios.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp -r /content/drive/MyDrive/AssignmentsIPCV/dataset.zip ./
!unzip dataset.zip

In [10]:
import os
import cv2
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt

## Configuration and Parameters
This cell contains all the parameters and feature flags that control the behavior of the book recognition algorithm:

In [11]:
# If True, perform a search over small rotation angles to refine the bounding box orientation against Canny edges
# Improves noticebly the accuracy for some instances, but worse a lot for others because of background edges (so, not enabled for the final version)
REFINE_ROTATION = False

# Apply CLAHE to model/scene images to reduce the effect of uneven illumination
# (This often helps keypoint detection on photographs taken under variable lighting conditions)
HANDLE_ILLUMINATION = True


# If True, visual intermediate images (matching points, refinement steps) for debugging purpose
PLOT_INTERMEDIATE_RESULTS = False

# If True, show the model image before showing the final scene results for debugging purpose
PLOT_MODEL_BEFORE_SCENE = False


# RANSAC reprojection pixel threshold used during findHomography, while the default value is 5, 
# experimentation found out that some projections were quite inaccurate, has been found that 
# 1 is a better value, leading to less false positives
RANSAC_REPROJ_TRESH = 1

# As per the RANSAC value, also the lowe's ratio value has been lowered, granting less false positives
LOWES_RATIO = 0.67

# Minimum matches required to attempt a detection
DETECTION_MIN_MATCH_COUNT = 25
ORIG_DETECTION_MIN_MATCH_COUNT = DETECTION_MIN_MATCH_COUNT

# After each detection, a mask is applied to the found instance (to try finding other instances in the same image), the 
# shrink is needed to avoid removing neighboring features related to other instances
OCCULTING_SHRINK_RATIO = 0.8

# As sometimes, the instances are rotated, a further search for best angle is done, small range 
# of angles are explored
ANGLE_RANGE_DEG = 15
ANGLE_STEP = 1

# Prevent infinite loops when iteratively masking detections in a scene
MAX_DETECTION_ATTEMPTS = 10

# IoU threshold for filtering duplicate detections, if IoU between two detections exceeds
# this value, the later one is discarded as a duplicate
IOU_THRESHOLD = 0.3

# Make behavior reproducible
cv2.setRNGSeed(42)
np.random.seed(42)

# SIFT detector for the instances:
# nfeatures=0 (return all detected features up to other constraints)
# nOctaveLayers=7 (increased the number of octaves to help with detecting features across a larger range of scales)
# contrastThreshold=0.02 (lower threshold yields more keypoints in low-contrast)
# edgeThreshold=15 (higher value retains more features along edges, useful for book spines)
# sigma=0.8 (halved from default value to preserves fine details, which is effective in combination with a higher nOctaveLayers to detect features on detailed book covers)
sift = cv2.SIFT_create(
    nfeatures=0,
    nOctaveLayers=7,
    edgeThreshold=15,
    sigma=0.8
)

## Helper Functions
This cell defines all the core utility functions for the book detection pipeline:
- kp_detection(): Detects SIFT keypoints and computes descriptors for an image
- detect_object(): Performs feature matching and homography estimation between model and scene
- refine_bbox_rotation(): Optimizes bounding box orientation using Canny edge alignment
- compute_iou(): Calculates intersection-over-union to avoid duplicate detections
- compute_quad_area(): Calculates the area of a quadrilateral bounding box
- get_labeled_corners(): Labels corners as top-left, top-right, bottom-left, bottom-right
- plot_rgb_image(): Helper for displaying images with matplotlib
- draw_matching_points(): Visualizes matched keypoints for debugging

In [ ]:
def kp_detection(img):
    """Detect SIFT keypoints and compute descriptors for the given image."""
    kp = sift.detect(img)
    
    img_kp, img_des = sift.compute(img, kp)
    return img_kp, img_des

def detect_object(model_kp, model_des, scene_kp, scene_des, min_match_count=4):
	""" Given keypoints and descriptors for a model and a scene, find matching
	keypoints and estimate a homography matrix if enough matches are found.
	"""
	# Finding the two best matches for each descriptor
	FLANN_INDEX_KDTREE = 1
	index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
	search_params = dict(checks=50, random_seed=42)  # Add random_seed for reproducibility
	flann = cv2.FlannBasedMatcher(index_params, search_params)
	matches = flann.knnMatch(model_des, scene_des, k=2)

	# Apply Lowe's ratio test on the matches
	good_matches = [m for m, n in matches if m.distance < LOWES_RATIO * n.distance]

	if len(good_matches) < min_match_count:
		# Not enough matches to continue
		return []

	detections = []

	# Prepare point arrays for homography estimation (model->scene)
	src_pts = np.float32([model_kp[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
	dst_pts = np.float32([scene_kp[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

	# Estimate homography with RANSAC to discard outliers among the matched keypoints
	M, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, RANSAC_REPROJ_TRESH)

	if M is not None:
		# Record number of matches used, homography, and the matches themselves
		detections.append((len(good_matches), M, good_matches))

	return detections


def refine_bbox_rotation(initial_corners, canny_edges, angle_range=15, angle_step=1):
    """This function performs a search over different degrees of rotation angles to select
    the angle whose polygon overlaps the most with the Canny edges of the scene.

    note: This assumes object boundaries correspond to strong Canny edges which is not always the case,
    indeed even if there are improvements in most of the instances, there are few cases where the result
    is worse than the original (mainly due to the fact that some rotations have overlapping edge points
    with edges not related to the instance, i.e. some other books, which leads to confusion).
    """
    best_score = -np.inf
    best_corners = initial_corners

    # Centroid of the bounding box (used as the rotation center)
    centroid = tuple(np.mean(initial_corners, axis=0))

    # Larger angle penalty factor
    perimeter_approx = 0
    for i in range(4):
        perimeter_approx += np.linalg.norm(initial_corners[i] - initial_corners[(i + 1) % 4])
    penalty_factor = perimeter_approx / 25.0

    # Trying a small range of rotations + computation of a simple edge-alignment score
    for angle in range(-angle_range, angle_range + 1, angle_step):
        rot_mat = cv2.getRotationMatrix2D(centroid, angle, 1.0)
        reshaped_corners = initial_corners.reshape(-1, 1, 2)
        rotated_corners = cv2.transform(reshaped_corners, rot_mat).reshape(4, 2)

        # Create a mask with the rotated polygon and count how many edge pixels
        # coincide with the polygon boundary. Thicker polylines help tolerate
        # small misalignments.
        mask = np.zeros_like(canny_edges)
        cv2.polylines(mask, [np.int32(rotated_corners)], isClosed=True, color=255, thickness=3)
        intersection = cv2.bitwise_and(mask, canny_edges)
        raw_edge_score = np.sum(intersection > 0)

        # Final score combines edge overlap and an angle penalty to prevent
        # arbitrary large rotations.
        score = raw_edge_score - (abs(angle) * penalty_factor)

        if score > best_score:
            best_score = score
            best_corners = rotated_corners

    return best_corners


def compute_iou(box1, box2):
    """Compute intersection-over-union between two bounding boxes, by rasterizing them,
    then computing the IoU pixel-wise.
    """
    h, w = 1000, 1000
    mask1 = np.zeros((h, w), dtype=np.uint8)
    mask2 = np.zeros((h, w), dtype=np.uint8)
    cv2.fillPoly(mask1, [np.int32(box1)], color=255)
    cv2.fillPoly(mask2, [np.int32(box2)], color=255)
    intersection = cv2.bitwise_and(mask1, mask2)
    union = cv2.bitwise_or(mask1, mask2)
    inter_area = np.sum(intersection > 0)
    union_area = np.sum(union > 0)
    if union_area == 0:
        return 0.0
    return inter_area / union_area


def compute_quad_area(corners):
	"""Compute the area of a quadrilateral defined by its corner points."""
	n = len(corners)
	area = 0.0
	for i in range(n):
		j = (i + 1) % n
		area += corners[i][0] * corners[j][1]
		area -= corners[j][0] * corners[i][1]
	area = abs(area) / 2.0
	return area


def get_labeled_corners(corners):
    """
    Label the corners of a quadrilateral defined by its corner points.
    """
    points = corners.tolist()
    points.sort(key=lambda p: p[1])
    top_points = points[:2]
    bottom_points = points[2:]
    top_points.sort(key=lambda p: p[0])
    bottom_points.sort(key=lambda p: p[0])
    tl = (int(round(top_points[0][0])), int(round(top_points[0][1])))
    tr = (int(round(top_points[1][0])), int(round(top_points[1][1])))
    bl = (int(round(bottom_points[0][0])), int(round(bottom_points[0][1])))
    br = (int(round(bottom_points[1][0])), int(round(bottom_points[1][1])))
    return {
        'top_left': tl,
        'top_right': tr,
        'bottom_left': bl,
        'bottom_right': br
    }


def plot_rgb_image(img_rgb, title=None):
    """Helper to plot RGB images"""
    plt.figure(figsize=(10, 8))
    plt.imshow(img_rgb)
    if title:
        plt.title(title)
    plt.axis('off')
    plt.show()


def draw_matching_points(scene_img_gray, scene_kp, good_matches):
    """Draw green circles at each matched scene keypoint for visualization"""
    scene_img = cv2.cvtColor(scene_img_gray, cv2.COLOR_GRAY2BGR)
    for match in good_matches:
        scene_pt = scene_kp[match.trainIdx].pt
        center = (int(scene_pt[0]), int(scene_pt[1]))
        cv2.circle(scene_img, center, radius=3, color=(0, 255, 0), thickness=2)
    return scene_img



## Data Preprocessing & Feature Extraction
This cell loads all model and scene images and preprocesses them for the detection pipeline:

Process:
1. Load Images: Reads all 22 model images and 29 scene images from the dataset
2. Apply CLAHE: Optionally applies Contrast Limited Adaptive Histogram Equalization to handle varying illumination conditions
3. Extract Features: Computes SIFT keypoints and descriptors for all images (done once for efficiency)
4. Generate Canny Edges: Creates edge maps for scenes (used in rotation refinement)

This preprocessing step optimizes the pipeline by computing features once rather than repeatedly during detection.

In [13]:
# Load images and scenes
img_models = [f"dataset/models/model_{n}.png" for n in range(22)]
img_scenes = [f"dataset/scenes/scene_{n}.jpg" for n in range(29)]

# Extract keypoints and descriptors from model images (optimization to do it once for all)
models_data = []
scenes_data = []

print("Computing model descriptors")
for model_path in img_models:
    img_model_gray = cv2.imread(model_path, cv2.IMREAD_GRAYSCALE)
    if img_model_gray is None:
        print(f"Problem loading model image {model_path}")
        continue

    # Optionally apply CLAHE to reduce illumination differences between model
    # photos and scene photos. This helps SIFT find more stable keypoints under
    # variable lighting.
    if HANDLE_ILLUMINATION:
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        img_model_gray = clahe.apply(img_model_gray)

    model_kp, model_des = kp_detection(img_model_gray)
    models_data.append((model_path, model_kp, model_des, img_model_gray))

# Extract keypoints, descriptors and Canny's edges from scene images (optimization to do it once for all)
print("Computing scene descriptors")
for scene_path in img_scenes:
    img_scene_gray = cv2.imread(scene_path, cv2.IMREAD_GRAYSCALE)
    if img_scene_gray is None:
        print(f"Problem loading scene image {scene_path}")
        continue

    if HANDLE_ILLUMINATION:
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        img_scene_gray = clahe.apply(img_scene_gray)

    scene_kp, scene_des = kp_detection(img_scene_gray)
    
    canny_scene = cv2.Canny(img_scene_gray, 50, 150)
    canny_scene = cv2.dilate(canny_scene, np.ones((3, 3), np.uint8))
    scenes_data.append((scene_path, scene_kp, scene_des, img_scene_gray, canny_scene))

Computing model descriptors
Computing scene descriptors
Computing scene descriptors


## Main Detection Pipeline
This is the core detection algorithm that identifies book instances in shelf scenes:
1. Model-Scene Matching: For each model, match it against all scenes using SIFT features
2. Iterative Detection: Use progressive masking to find multiple instances of the same book
3. Homography Estimation: Transform model coordinates to scene coordinates using RANSAC
4. Bounding Box Refinement: Optionally refine orientation using Canny edge alignment
5. Duplicate Filtering: Use IoU thresholding to avoid detecting the same instance multiple times
6. Masking: Mask detected regions to enable finding additional instances
7. Results Collection: Store detection coordinates, areas, and generate visualization images

The algorithm generates detailed reports for each book with instance counts, coordinates, and areas.

In [15]:
book_detections = defaultdict(list)

for model_path, model_kp, model_des, model_img in models_data:
    print(f"\nProcessing {model_path.split('/')[-1]}:")
    for scene_path, scene_kp, scene_des, scene_img_gray_mutable, canny_scene in scenes_data:
        # As some of the scene is going to be masked, to be able to find multiple instances, we 
        # work on copies of keypoints/descriptors
        current_scene_kp = scene_kp
        current_scene_des = scene_des

        original_scene_img = cv2.imread(scene_path, cv2.IMREAD_GRAYSCALE)
        if HANDLE_ILLUMINATION:
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
            original_scene_img = clahe.apply(original_scene_img)
        scene_img_gray_for_masking = scene_img_gray_mutable.copy()

        detection_count = 0
        all_detections = []
        current_min_matches = DETECTION_MIN_MATCH_COUNT
        attempt_count = 0

        # Iteratively detect and mask found instances until no more are found or
        # we reach a safety attempt limit to avoid possible infinite loops
        while True:
            attempt_count += 1
            if attempt_count > MAX_DETECTION_ATTEMPTS:
                #print(f"\t\tMax detection attempts reached for {scene_path.split('/')[-1]}. Stopping.")
                break

            detections = detect_object(
                model_kp, model_des, current_scene_kp, current_scene_des,
                min_match_count=current_min_matches
            )

            # If no more detections in the current (masked) scene
            if not detections:
                break

            # Keep polygons to mask after processing all clusters (so clusters do
            # not interfere with each other within the same iteration)
            to_mask = []
            
            for det in detections:
                n_matches, M, good_matches = det

                # Optional visualization of matched scene keypoints
                if PLOT_INTERMEDIATE_RESULTS:
                    scene_with_points = draw_matching_points(original_scene_img, current_scene_kp, good_matches)
                    plot_rgb_image(cv2.cvtColor(scene_with_points, cv2.COLOR_BGR2RGB), 
								f"Matching Points in {scene_path.split('/')[-1]}")

                # Build the model's bounding rectangle and project it into the scene
                h, w = model_img.shape[:2]
                model_corners = np.float32([[0, 0], [w, 0], [w, h], [0, h]]).reshape(-1, 1, 2)
                initial_scene_corners = cv2.perspectiveTransform(model_corners, M).reshape(4, 2)

                # Optionally box refinement using image edges
                if REFINE_ROTATION:
                    print("\t\tRefining bounding box rotation...")
                    final_scene_corners = refine_bbox_rotation(
                        initial_scene_corners, canny_scene,
                        ANGLE_RANGE_DEG, ANGLE_STEP
                    )
                else:
                    final_scene_corners = initial_scene_corners

                # Check overlap with previously accepted detections using IoU function, if 
                # the IoU is high enough we skip this detection to avoid duplicates
                overlap = False
                for prev_box in all_detections:
                    if compute_iou(final_scene_corners, prev_box) > IOU_THRESHOLD:
                        if PLOT_INTERMEDIATE_RESULTS:
                            print(f"\t\tSkipping detection due to high overlap with previous.")
                        
                        overlap = True
                        break

                if not overlap:
                    detection_count += 1
                    print(f"\t{scene_path.split('/')[-1]} - Detection {detection_count}: {n_matches} good matches")
                    all_detections.append(final_scene_corners)
                    to_mask.append(final_scene_corners)

                # Visualization comparison between initial and refined box
                if PLOT_INTERMEDIATE_RESULTS and REFINE_ROTATION:
                        vis_img = cv2.cvtColor(original_scene_img, cv2.COLOR_GRAY2BGR)
                        cv2.polylines(vis_img, [np.int32(initial_scene_corners)], isClosed=True, color=(0, 0, 255), thickness=2)
                        cv2.polylines(vis_img, [np.int32(final_scene_corners)], isClosed=True, color=(0, 255, 0), thickness=3)
                        plot_rgb_image(cv2.cvtColor(vis_img, cv2.COLOR_BGR2RGB), f"Rotation Refinement (Red: Initial, Green: Refined) in {scene_path.split('/')[-1]}")

            # After the processing of the instance, mask it from the scene image such that
            # further iterations will not rediscover the same instances
            # note: as part of the found image might be useful for another very close detection,
            # a smaller part of the image is occulted instead (defined by OCCULTING_SHRINK_RATIO variable)
            for corners in to_mask:
                centroid = np.mean(corners, axis=0)
                mask_corners = centroid + OCCULTING_SHRINK_RATIO * (corners - centroid)
                mask_polygon = np.int32([mask_corners])
                cv2.fillPoly(scene_img_gray_for_masking, mask_polygon, color=0)

            # Recompute keypoints/descriptors on the masked image for the next iteration
            current_scene_kp, current_scene_des = kp_detection(scene_img_gray_for_masking)

        # If we found at least one detection, save/plot combined results
        if detection_count > 0:
            if PLOT_MODEL_BEFORE_SCENE:
                plot_rgb_image(cv2.cvtColor(model_img, cv2.COLOR_BGR2RGB),
                               f"Model Image: {model_path.split('/')[-1]}")
                
                plot_rgb_image(cv2.cvtColor(final_img, cv2.COLOR_BGR2RGB),
							f"All Detections in {scene_path.split('/')[-1]}")
    
            final_img = cv2.cvtColor(original_scene_img, cv2.COLOR_GRAY2BGR)
            for box in all_detections:
                cv2.polylines(final_img, [np.int32(box)], isClosed=True, color=(0, 255, 0), thickness=3)

            # Save final visualization for bounding box visualization 
            # Filenames contain the model and scene identifiers plus how many instances were detected.
            os.makedirs("final_detections", exist_ok=True)
            model_name = (model_path.split('/')[-1]).split('.')[0]
            scene_name = (scene_path.split('/')[-1]).split('.')[0]
            cv2.imwrite(f"final_detections/{model_name}_{scene_name}_x{detection_count}.png", final_img)

            book_id = model_name.split('_')[1]
            for box in all_detections:
                labeled = get_labeled_corners(box)
                area = compute_quad_area(box)
                book_detections[book_id].append((scene_name, labeled, area))

    # After processing all scenes for this model, print the aggregated report
    book_id = model_name.split('_')[1]
    if book_detections[book_id]:
        total = len(book_detections[book_id])
        print(f"Book {book_id} - {total} instance(s) found:")
        for i, (scene, labeled, area) in enumerate(book_detections[book_id], 1):
            tl = labeled['top_left']
            tr = labeled['top_right']
            bl = labeled['bottom_left']
            br = labeled['bottom_right']
            print(f"Instance {i} in {scene} {{top_left: ({tl[0]}, {tl[1]}), top_right: ({tr[0]}, {tr[1]}), bottom_left: ({bl[0]}, {bl[1]}), bottom_right: ({br[0]}, {br[1]}), area: {int(area)}px}}")


Processing model_0.png:
	scene_26.jpg - Detection 1: 400 good matches
	scene_26.jpg - Detection 2: 197 good matches
Book 0 - 2 instance(s) found:
Instance 1 in scene_26 {top_left: (210, 181), top_right: (244, 181), bottom_left: (210, 568), bottom_right: (244, 568), area: 13235px}
Instance 2 in scene_26 {top_left: (170, 181), top_right: (205, 181), bottom_left: (176, 569), bottom_right: (210, 571), area: 13486px}

Processing model_1.png:
	scene_28.jpg - Detection 1: 174 good matches
	scene_28.jpg - Detection 2: 78 good matches
Book 1 - 2 instance(s) found:
Instance 1 in scene_28 {top_left: (207, 248), top_right: (236, 248), bottom_left: (207, 554), bottom_right: (236, 555), area: 8846px}
Instance 2 in scene_28 {top_left: (237, 248), top_right: (266, 244), bottom_left: (237, 551), bottom_right: (266, 552), area: 8878px}

Processing model_2.png:
	scene_27.jpg - Detection 1: 443 good matches
Book 2 - 1 instance(s) found:
Instance 1 in scene_27 {top_left: (258, 146), top_right: (303, 146),

## Previous Attempts
As the task ask for the project to be coincisive all of the tries with different techniques have been deleted, but here is a list of what has been tried and the encountered problems:
- Template Matching: different tecniques of template matching has been tried, among which ZNCC gave the best results, especially in scene/models with intensity changes, however it struggled with different scale models like scene_27, and other cases
- DBSCAN: During the exploration to improve SIFT method, DBSCAN has been used after the keypoints matching phase, as sometimes there were keypoints really off, from the actual model, using DBSCAN, in particular its property of eliminating outliers helped generating better homographies, however the cluster of points were different for some pairs of model/scene, which led to be impossible to find the best parameter, then, by tuning more accuratelty parameters of the SIFT model the outliers points problem has been solved anyway, without the necessity of DBSCAN.
- Hough Transform: Another exploration to improve the SIFT method went through the usage of Hough transform, in particular, the idea was to pre-compute for each scene all possible book instances (of rectangular shape), and then run a simpler SIFT on the found rectangular shapes